<h1>Purpose of this notebook</h1>
The purpose of this notebook is to see the effects of data augmentation.

Data augmentation is when the data is augmented to generate multiple versions of the same image.

When only a small dataset of images is available it might show to be beneficial to use data augmentation.
Since the neural network might be overfitting the training data.

In [6]:
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.applications import mobilenet
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.models import Model, Sequential
from keras import models
from keras import layers

<b>Resources used to train, validate and test.</b>

In [7]:
train_path = '../Retraining/resources/train'
valid_path = '../Retraining/resources/valid'
test_path = '../Retraining/resources/test'

<h1>ImageDataGenerators: Without data augmentation</h1>


In [8]:
batch_size = 16

train_datagen_wo = ImageDataGenerator(
        rescale=1./255
)

valid_datagen_wo = ImageDataGenerator(
        rescale=1./255
)

# Train data generator
train_generator_wo = train_datagen_wo.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=batch_size,
        classes=['potato', 'catfood', 'table'])

# Validation data generator
validation_generator_wo = valid_datagen_wo.flow_from_directory(
        valid_path,
        target_size=(224, 224),
        batch_size=batch_size,
        classes=['potato', 'catfood', 'table'])

Found 299 images belonging to 3 classes.
Found 82 images belonging to 3 classes.


<h1>ImageDataGenerators: With data augmentation</h1>
<ul>
    <li> <b>Rescale</b> rescales image data from range [0 255] to range [0 1] </li>
    <li><b>Shear_range</b> a range with how much the data is augmented with respect to shear</li>
    <li><b>Zoom_range</b> range for a random zoom. [1-zoom_range 1+zoom_range]</li>
    <li><b>Horizontal_flip:</b> boolean if it should be possible to make a random horizontal flip</li>
</ul>


Should be tested:
<ul>
    <li><b>Rotation_range</b></li>
    <li><b>brightness_range</b></li>
</ul>

In [9]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

valid_datagen = ImageDataGenerator(
        rescale=1./255
)

# Train data generator
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=batch_size,
        classes=['potato', 'catfood', 'table'])

# Validation data generator
validation_generator = valid_datagen.flow_from_directory(
        valid_path,
        target_size=(224, 224),
        batch_size=batch_size,
        classes=['potato', 'catfood', 'table'])

Found 299 images belonging to 3 classes.
Found 82 images belonging to 3 classes.


<h1>Neural Network</h1>

In [10]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
activation_2 (Activation)    (None, 109, 109, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 186624)            0         
__________

<h1>Utility functions</h1>

In [11]:
def plot_history(history):
    #print(history.history.keys())
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

<h1>Testing performance with data augmentation</h1>

In [12]:
history = model.fit_generator(
            train_generator,
            steps_per_epoch=299 // batch_size,
            epochs=50,
            validation_data=validation_generator,
            validation_steps=82 // batch_size)

Epoch 1/50
18/18 [==============================] - 13s 740ms/step - loss: 1.7101 - acc: 0.5312 - val_loss: 0.6239 - val_acc: 0.6220
Epoch 2/50
18/18 [==============================] - 3s 158ms/step - loss: 0.5864 - acc: 0.6899 - val_loss: 0.3759 - val_acc: 0.8537
Epoch 3/50
18/18 [==============================] - 3s 161ms/step - loss: 0.6614 - acc: 0.7205 - val_loss: 0.3591 - val_acc: 0.8902
Epoch 4/50
18/18 [==============================] - 3s 172ms/step - loss: 0.4818 - acc: 0.7880 - val_loss: 0.4025 - val_acc: 0.8415
Epoch 5/50
18/18 [==============================] - 3s 191ms/step - loss: 0.4425 - acc: 0.8019 - val_loss: 0.3810 - val_acc: 0.8415
Epoch 6/50
18/18 [==============================] - 3s 167ms/step - loss: 0.3432 - acc: 0.8093 - val_loss: 0.5983 - val_acc: 0.7683
Epoch 7/50
18/18 [==============================] - 3s 152ms/step - loss: 0.4004 - acc: 0.8133 - val_loss: 0.3452 - val_acc: 0.8659
Epoch 8/50
18/18 [==============================] - 3s 146ms/step - loss: 0

In [13]:
test_datagen = ImageDataGenerator(
        rescale=1./255
)

# this is a similar generator, for validation data
test_generator = valid_datagen.flow_from_directory(
        test_path,
        target_size=(224, 224),
        batch_size=batch_size,
        classes=['potato', 'catfood', 'table'])

Found 35 images belonging to 3 classes.


In [14]:
imgs, labels = next(test_generator)

#ynew = model.predict(imgs)

model.evaluate(imgs,labels, batch_size=batch_size)

16/16 [==============================] - 0s 2ms/step


[0.2578544020652771, 0.875]

In [15]:
gul_path = '../Retraining/tfpoet/tf_files/test/gulerod.jpg'
img = image.load_img(gul_path, target_size=(224, 224))
img = image.img_to_array(img)
img = img
img = np.expand_dims(img, axis=0)
out = model.predict(img)

FileNotFoundError: [Errno 2] No such file or directory: '../Retraining/tfpoet/tf_files/test/gulerod.jpg'

In [ ]:
labels

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

<h1>Testing performance without data augmentation</h1>

In [ ]:
history = model.fit_generator(
            train_generator_wo,
            steps_per_epoch=299 // batch_size,
            epochs=50,
            validation_data=validation_generator_wo,
            validation_steps=82 // batch_size)

In [ ]:
imgs, labels = next(test_generator)

#ynew = model.predict(imgs)

model.evaluate(imgs,labels, batch_size=batch_size)

In [ ]:
plot_history(history)

<h1>Testing with original image size: 1280x720</h1>

In [ ]:
train_path = '../potato_and_catfood/train'
valid_path = '../potato_and_catfood/valid'
test_path = '../potato_and_catfood/test'